In [1]:
import pymorphy2
import re
import string
import gensim
import pandas as pd
from pymorphy2 import MorphAnalyzer
from collections import Counter
from nltk.corpus import stopwords
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
morph = MorphAnalyzer()

In [2]:
stops = set(stopwords.words('russian')) | {'gt', 'int', 'if', 'x', 'to', 'in', 'return', 'case', 'i', '•', 
                                          'null', 'set', 'as', 'from', 'var', 'lt', 'on', 'is', 'end', 'b',
                                          'else', 'bool', 'const', 'function', 'class', 'string', 'import',
                                          'then', 'type', 'person', 'select', 'def', 'up', 'down', 'active',
                                          'print', 'assert', 'j', 'str', 'loop', 'char', 'unsigned', 'true',
                                          'main', 'func'}

In [3]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [4]:
texts = open('/Users/macbookpro/Downloads/habr.txt').read().splitlines()

In [5]:
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

In [6]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.7) # threshold можно подбирать

In [7]:
p = gensim.models.phrases.Phraser(ph)

In [8]:
ngrammed_texts = p[texts]

In [9]:
words = gensim.corpora.Dictionary(ngrammed_texts)

In [10]:
words.filter_extremes(no_above=0.15, no_below=22)
words.compactify()

In [11]:
print(words)

Dictionary(6710 unique tokens: ['2-х', '3.0', 'address', 'api', 'architecture']...)


In [22]:
tfidf = gensim.models.TfidfModel(corpus, id2word=words)

In [23]:
corpus = tfidf[corpus]

In [13]:
corpus = [words.doc2bow(text) for text in ngrammed_texts]

In [14]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=words, eval_every=0)

In [15]:
lda.print_topics()

[(90,
  '0.005*"скидка" + 0.004*"apple" + 0.004*"книга" + 0.004*"цена" + 0.003*"000" + 0.003*"стоимость" + 0.003*"формула" + 0.003*"энергия" + 0.003*"товар" + 0.003*"рейтинг"'),
 (69,
  '0.006*"игра" + 0.005*"адрес" + 0.005*"услуга" + 0.004*"add" + 0.003*"процессор" + 0.003*"виртуальный" + 0.003*"выделить" + 0.003*"трафик" + 0.003*"хост" + 0.002*"участник"'),
 (2,
  '0.003*"процессор" + 0.002*"сотрудник" + 0.002*"виртуальный" + 0.002*"ключ" + 0.002*"user" + 0.002*"адрес" + 0.002*"услуга" + 0.002*"операция" + 0.002*"экран" + 0.002*"ядро"'),
 (83,
  '0.005*"microsoft" + 0.002*"таблица" + 0.002*"виртуальный" + 0.002*"страна" + 0.002*"трафик" + 0.002*"программирование" + 0.002*"this" + 0.002*"go" + 0.002*"локальный" + 0.002*"производительность"'),
 (98,
  '0.004*"user" + 0.004*"вселенная" + 0.004*"телефон" + 0.003*"звук" + 0.002*"таблица" + 0.002*"тело" + 0.002*"n" + 0.002*"свет" + 0.002*"постоянный" + 0.002*"дерево"'),
 (87,
  '0.006*"игра" + 0.005*"тест" + 0.004*"диск" + 0.003*"текстура"

In [16]:
lda = gensim.models.LdaMulticore(corpus, 200, id2word=words, passes=3, alpha='asymmetric', eval_every=0)

In [17]:
lda.print_topics()

[(197,
  '0.005*"дизайн" + 0.004*"частица" + 0.004*"теория" + 0.003*"игра" + 0.003*"здание" + 0.003*"изображение" + 0.003*"квантовый" + 0.002*"автомат" + 0.002*"движение" + 0.002*"построить"'),
 (193,
  '0.050*"byte" + 0.010*"комната" + 0.009*"сканер" + 0.009*"void" + 0.007*"температура" + 0.006*"меню" + 0.006*"поверхность" + 0.006*"amp;&amp" + 0.006*"9" + 0.005*"output"'),
 (191,
  '0.008*"виртуальный" + 0.008*"станция" + 0.007*"звук" + 0.006*"скрипт" + 0.006*"базовый" + 0.004*"плагин" + 0.004*"температура" + 0.004*"офис" + 0.003*"api" + 0.003*"сетевой"'),
 (192,
  '0.029*"вм" + 0.019*"виртуальный" + 0.015*"linux" + 0.015*"хост" + 0.013*"windows" + 0.012*"транзакция" + 0.010*"защита" + 0.010*"схд" + 0.009*"ос" + 0.008*"драйвер"'),
 (198,
  '0.015*"узел" + 0.014*"android" + 0.009*"экран" + 0.009*"university" + 0.008*"кластер" + 0.007*"вкладка" + 0.006*"systems" + 0.005*"data" + 0.005*"at" + 0.005*"cs"'),
 (199,
  '0.008*"лицензия" + 0.008*"тест" + 0.006*"бизнес" + 0.005*"тестирование" 

In [18]:
lda = gensim.models.LdaModel(corpus, 150, id2word=words, passes=6, alpha='auto', eval_every=0)

In [19]:
lda.print_topics()

[(5,
  '0.249*"нажать" + 0.159*"галактика" + 0.128*"тёмный" + 0.044*"кнопка" + 0.040*"тёмный_материя" + 0.031*"открытие" + 0.029*"горячее" + 0.025*"иначе" + 0.023*"центр" + 0.014*"объяснение"'),
 (146,
  '0.200*"усилитель" + 0.167*"микросхема" + 0.162*"искажение" + 0.091*"подбирать" + 0.061*"эффект" + 0.036*"клон" + 0.034*"повредить" + 0.033*"обратный" + 0.029*"спринт" + 0.027*"когнитивный"'),
 (4,
  '0.730*"символ" + 0.087*"шрифт" + 0.048*"валюта" + 0.036*"зуб" + 0.013*"unicode" + 0.008*"составной" + 0.005*"избегать" + 0.004*"нагрузка" + 0.003*"снижать" + 0.003*"нормальный"'),
 (90,
  '0.256*"вода" + 0.088*"gitlab" + 0.076*"вод" + 0.054*"script" + 0.049*"температура" + 0.038*"лёд" + 0.034*"кислород" + 0.030*"ci" + 0.027*"pages" + 0.027*"гелий"'),
 (141,
  '0.333*"фотография" + 0.108*"разрешение" + 0.057*"горизонт" + 0.050*"снимать" + 0.042*"кадр" + 0.041*"проекция" + 0.040*"съёмка" + 0.025*"телескоп" + 0.024*"фото" + 0.021*"видный"'),
 (129,
  '0.216*"право" + 0.110*"механизм" + 0.075

In [20]:
topics = [lda.print_topics()]

In [21]:
print(topics[0][0], '\n\n', topics[0][4], '\n\n', topics[0][17])

(5, '0.249*"нажать" + 0.159*"галактика" + 0.128*"тёмный" + 0.044*"кнопка" + 0.040*"тёмный_материя" + 0.031*"открытие" + 0.029*"горячее" + 0.025*"иначе" + 0.023*"центр" + 0.014*"объяснение"') 

 (24, '0.178*"test" + 0.076*"read" + 0.072*"write" + 0.069*"bytes" + 0.061*"date" + 0.049*"t" + 0.042*"архитектор" + 0.034*"server" + 0.033*"os" + 0.031*"sleep"') 

 (63, '0.013*"энергия" + 0.010*"теория" + 0.009*"университет" + 0.009*"научный" + 0.007*"учёный" + 0.005*"мировой" + 0.005*"наука" + 0.005*"институт" + 0.005*"центр" + 0.005*"технологический"')


модели на преобразованном корпусе:

In [24]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=words, eval_every=0)

In [25]:
lda.print_topics()

[(79,
  '0.004*"игра" + 0.003*"microsoft" + 0.002*"уязвимость" + 0.002*"скидка" + 0.002*"windows" + 0.002*"кожа" + 0.002*"google" + 0.002*"сотрудник" + 0.002*"ноутбук" + 0.002*"программист"'),
 (71,
  '0.003*"ия" + 0.003*"игра" + 0.003*"ядро" + 0.002*"программист" + 0.002*"умение" + 0.002*"учёный" + 0.002*"соревнование" + 0.002*"российский" + 0.002*"предприниматель" + 0.002*"программирование"'),
 (7,
  '0.004*"игра" + 0.004*"сигнал" + 0.003*"усилитель" + 0.002*"игрок" + 0.002*"переменный" + 0.002*"метка" + 0.002*"бот" + 0.002*"гибкость" + 0.002*"документ" + 0.002*"наушник"'),
 (73,
  '0.003*"api" + 0.003*"игра" + 0.002*"изображение" + 0.002*"кристалл" + 0.002*"корпорация" + 0.002*"android" + 0.002*"контроллер" + 0.002*"бизнес" + 0.002*"кнопка" + 0.002*"iot"'),
 (32,
  '0.002*"windows" + 0.002*"бот" + 0.002*"трафик" + 0.002*"наушник" + 0.002*"сотрудник" + 0.002*"профессиональный" + 0.002*"игрок" + 0.002*"канал" + 0.002*"google" + 0.002*"документ"'),
 (24,
  '0.003*"тест" + 0.002*"false"

In [26]:
lda = gensim.models.LdaMulticore(corpus, 200, id2word=words, passes=3, alpha='asymmetric', eval_every=0)

In [27]:
lda.print_topics()

[(195,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (196,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (198,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (194,
  '0.076*"переменный" + 0.074*"символ" + 0.050*"выражение" + 0.045*"массив" + 0.043*"аргумент" + 0.035*"синтаксис" + 0.033*"foo" + 0.020*"оператор" + 0.018*"go" + 0.015*"преобразование"'),
 (199,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (191,
  '0.000*"по-хорошему

In [28]:
lda = gensim.models.LdaModel(corpus, 150, id2word=words, passes=6, alpha='auto', eval_every=0)

In [29]:
lda.print_topics()

[(123,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (97,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (90,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (71,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"'),
 (24,
  '0.014*"интуиция" + 0.000*"предпочтение" + 0.000*"работник" + 0.000*"характеристика" + 0.000*"принятие" + 0.000*"тест" + 0.000*"иб" + 0.000*"психологический" + 0.000*"личность" + 0.000*"деловой"'),
 (134,
  '0.000*"по-х

In [30]:
lda = gensim.models.LdaModel(corpus, 85, id2word=words, passes=1, alpha='auto', eval_every=0)

In [31]:
lda.print_topics()

[(32,
  '0.012*"signal" + 0.008*"ssd" + 0.005*"basic" + 0.005*"tor" + 0.004*"ген" + 0.004*"отклик" + 0.003*"смерть" + 0.002*"клетка" + 0.002*"диск" + 0.002*"одновременный"'),
 (22,
  '0.009*"болезнь" + 0.007*"звонок" + 0.005*"номер" + 0.005*"one" + 0.005*"разрешать" + 0.005*"заболевание" + 0.004*"drive" + 0.004*"больной" + 0.003*"диагноз" + 0.003*"вещество"'),
 (61,
  '0.004*"safari" + 0.004*"webpack" + 0.004*"пространство" + 0.003*"text" + 0.003*"render" + 0.002*"grep" + 0.002*"район" + 0.002*"калибровка" + 0.002*"шрифт" + 0.002*"политика"'),
 (45,
  '0.007*"3d-печать" + 0.004*"hash" + 0.004*"3d-принтер" + 0.004*"школа" + 0.003*"регистратор" + 0.003*"void" + 0.003*"производство" + 0.003*"super" + 0.002*"печатать" + 0.002*"p"'),
 (34,
  '0.011*"стек" + 0.010*"php" + 0.006*"ddr4" + 0.005*"вт" + 0.004*"visual" + 0.004*"носитель" + 0.003*"беспилотный" + 0.003*"джон" + 0.003*"дрон" + 0.003*"visual_studio"'),
 (21,
  '0.007*"pass" + 0.006*"платёж" + 0.005*"курс" + 0.005*"погода" + 0.004*"ru

Первое, что бросилось в глаза после применения tf-idf — это появление множества дубликатов на нескольких вариантах настроек модели, вот таких: 

(31,
  '0.000*"по-хорошему" + 0.000*"погружение" + 0.000*"морской" + 0.000*"отверстие" + 0.000*"отметка" + 0.000*"капля" + 0.000*"повредить" + 0.000*"58" + 0.000*"удар" + 0.000*"испытание"')
  
Вероятно, это как-то связано с количеством проходов по корпусу, но до tf-idf этого не было.

При этом сами темы стали более осмысленными — если однозначную тематику угадать все еще можно не всегда, то связи между словами в теме почти всегда можно найти, в отличие от результатов моделей до tf-idf:

(7,
  '0.048*"сигнал" + 0.035*"температура" + 0.031*"датчик" + 0.030*"питание" + 0.028*"корпус" + 0.024*"напряжение" + 0.024*"плата" + 0.021*"модуль" + 0.020*"ток" + 0.019*"диск"') – про hardware
  
 (6,
  '0.084*"блокчейн" + 0.052*"вакансия" + 0.038*"кандидат" + 0.038*"рейтинг" + 0.031*"аудитория" + 0.026*"bitcoin" + 0.022*"представитель" + 0.019*"стартап" + 0.017*"ценность" + 0.016*"город"') – про работу в блокчейне

Некоторые из тем ухудшились, например, в более ранней 

(56,
  '0.017*"автомобиль" + 0.016*"робот" + 0.010*"рубль" + 0.009*"телефон" + 0.009*"обзор" + 0.008*"производитель" + 0.007*"цена" + 0.007*"номер" + 0.006*"купить" + 0.006*"покупка"')
  
легче понять, о чем речь, нежели в более поздней

(5,
  '0.125*"автомобиль" + 0.042*"raspberry_pi" + 0.032*"дорога" + 0.020*"килограмм" + 0.013*"производство" + 0.012*"pi" + 0.011*"весить" + 0.011*"microsd" + 0.010*"аккумулятор" + 0.008*"наклон"')

.

In [32]:
stexts = [' '.join(text) for text in ngrammed_texts]

In [33]:
vectorizer = TfidfVectorizer(max_features=500, min_df=5, max_df=0.7, ngram_range=(1,3))

In [34]:
X = vectorizer.fit_transform(stexts)

In [35]:
model = NMF(n_components=30)

In [36]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [37]:
feat_names = vectorizer.get_feature_names()

In [38]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 то  очень  какой  делать
1 файл  скрипт  пакет  настройка
2 игра  игрок  разработчик  уровень
3 сеть  связь  интернет  оператор
4 система  управление  безопасность  работа
5 lt  gt  компонент  элемент
6 приложение  мобильный  android  разработка
7 устройство  смартфон  производитель  телефон
8 год  2016  миллион  стать
9 public  new  void  класс
10 компания  сотрудник  рынок  бизнес
11 the  of  and  for
12 код  функция  переменный  значение
13 сервер  клиент  виртуальный  машина
14 сайт  страница  браузер  адрес
15 проект  команда  разработчик  разработка
16 объект  элемент  класс  метод
17 дать  база  запрос  база дать
18 технология  задача  решение  материал
19 тест  тестирование  результат  производительность
20 сигнал  связь  канал  значение
21 google  android  сервис  api
22 windows  microsoft  linux  версия
23 камера  видео  смартфон  изображение
24 amp  for  void  gt
25 язык  программирование  программист  книга
26 человек  исследование  жизнь  работа
27 пользователь  сообщени

In [39]:
model.reconstruction_err_

49.74329980613673

In [40]:
vectorizer = TfidfVectorizer(max_features=1500, min_df=8, max_df=0.5, ngram_range=(1,3))

In [41]:
X = vectorizer.fit_transform(stexts)

In [42]:
model = NMF(n_components=30)

In [43]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [44]:
feat_names = vectorizer.get_feature_names()

In [45]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 человек  книга  какой  какой то
1 файл  папка  скрипт  строка
2 игра  игрок  игровой  играть
3 сигнал  звук  частота  напряжение
4 устройство  камера  смартфон  экран
5 компания  сотрудник  рынок  бизнес
6 приложение  android  мобильный  пользователь
7 lt  gt  div  lt div
8 public  new  void  private
9 the  of  and  for
10 человек  учёный  мозг  исследование
11 космический  спутник  орбита  аппарат
12 сайт  страница  пользователь  реклама
13 сервер  запрос  база  сервис
14 уязвимость  атака  безопасность  пароль
15 windows  linux  microsoft  виртуальный
16 печать  принтер  3d  материал
17 доклад  конференция  участник  тема
18 функция  объект  значение  метод
19 язык  код  программирование  программа
20 css  javascript  react  js
21 тест  тестирование  код  тестировать
22 товар  рубль  цена  скидка
23 amp  amp amp  echo  for
24 бот  сообщение  пользователь  канал
25 дата центр  центр  дата  оборудование
26 вселенная  энергия  звезда  свет
27 проект  команда  задача  разработка
28 сет

In [46]:
model.reconstruction_err_

55.24894214850625

In [47]:
vectorizer = TfidfVectorizer(max_features=1200, min_df=21, max_df=0.35, ngram_range=(1,3))

In [48]:
X = vectorizer.fit_transform(stexts)

In [49]:
model = NMF(n_components=30)

In [50]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [51]:
feat_names = vectorizer.get_feature_names()

In [52]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 книга  что  какой то  что то
1 файл  папка  скрипт  команда
2 игра  игрок  игровой  играть
3 функция  объект  значение  метод
4 устройство  камера  смартфон  экран
5 приложение  android  мобильный  ios
6 lt  gt  div  lt div
7 public  new  void  private
8 windows  microsoft  linux  ядро
9 товар  рубль  цена  скидка
10 космический  земля  спутник  аппарат
11 сайт  страница  реклама  браузер
12 атака  уязвимость  безопасность  пароль
13 the  of  and  for
14 доклад  конференция  участник  тема
15 рынок  технология  бизнес  продукт
16 учёный  мозг  клетка  исследование
17 css  javascript  react  js
18 печать  принтер  3d  материал
19 язык  программирование  программа  программист
20 запрос  база  таблица  база дать
21 дата центр  центр  дата  оборудование
22 amp  amp amp  for  echo
23 сервер  виртуальный  машина  виртуальный машина
24 бот  сообщение  канал  google
25 клиент  услуга  письмо  бизнес
26 тест  тестирование  команда  тестовый
27 сигнал  звук  частота  напряжение
28 сеть  трафи

In [53]:
model.reconstruction_err_

55.12926050688545

лучшей оказалась первая модель.

In [54]:
top_words = model.components_.argsort()[:,:-5:-1]
ready = []
for i in range(top_words.shape[0]):
    ready.append([feat_names[j] for j in top_words[i]])

In [125]:
print(ready[2], ready[3], ready[6])

['игра', 'игрок', 'разработчик', 'уровень'] ['приложение', 'мобильный', 'android', 'разработка'] ['проект', 'команда', 'разработчик', 'разработка']


осмысленности в результатах куда больше, чем в LDA, четко прослеживаются связи между словами и наличие общих тем — в данном случае это игры, мобильная разработка, проектная работа в разработке.